In [1]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd
import aesara.tensor as at
from aesara.tensor.subtensor import set_subtensor

%load_ext lab_black
%load_ext watermark

# Wine classification

Adapted from [unit 10: italywines123.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/italywines123.odc).

Data can be found [here](https://archive.ics.uci.edu/ml/datasets/wine).

Associated lecture video: Unit 10 Lesson 5

## Problem statement

This popular data set was provided by Forina et al (1988). 
The data below consist of results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines. 

  The variables are
  
   Column   Variable   Description  
   
  ==========================
  
     1      Y     Type (Response, 1,2,3)  
                      1 == 1 0 0;    2 == 0 1 0;   3 == 0 0 1
                      
     2      X1    Alcohol 
      
 	 3      X2    Malic acid 
     
 	 4      X3    Ash 
     
	 5      X4    Alcalinity of ash 
      
  	 6      X5    Magnesium 
    
	 7      X6    Total phenols 
      
 	 8      X7    Flavanoids 
     
     9      X8    Nonflavanoid phenols 
      
     10     X9    Proanthocyanins 
     
     11     X10  Color intensity 
     
     12     X11  Hue 
     
     13     X12 OD280/OD315 of diluted wines 
     
     14     X13  Proline 
 



(a) Fit the multinomial regression
that predicts the type of wine Y from predictors  X1 - X13.
What are estimated coefficients? What is the deviance?

(b) What is your prediction for  pp=P(Ynew=1)  if a new case has attributes  
 XX1 = 12.9, XX2 = 2,  XX3 = 2.4, XX4 = 17,
 XX5 = 100,  XX6 = 2.8, XX7 = 2.1, XX8 =  0.35,  XX9 = 1.6, XX10 = 5, 
XX11 = 1.05,  XX12 = 3, and XX13 = 750?
How would you classify this wine type, as 1, 2, or 3?

 
 Forina, M.,   Leardi, R.,   Armanino, C., and  Lanteri, S. (1988).
 PARVUS: An extendable package of programs for data exploration, classification and correlation,  Elsevier, Amsterdam,   ISBN 0-444-43012-1;
 also, Report at Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno,
  16147 Genoa, Italy.

In [2]:
data = pd.read_csv("../data/wine.data", header=None)
Y = pd.get_dummies(data[0]).to_numpy()
X = data.drop(0, axis=1).to_numpy()
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

In [3]:
X_aug.shape, Y.shape

((178, 14), (178, 3))

In [4]:
with pm.Model() as m:
    X_data = pm.Data("X", X_aug, mutable=True)
    Y_data = pm.Data("y", Y, mutable=False)
    _b = pm.Normal("_b", 0, tau=0.05, shape=(14, 2))
    b = at.concatenate([at.zeros((14, 1)), _b], axis=1)

    phi = pm.math.exp(pm.math.dot(X_data, b))
    # probabilities must sum to 1
    P = phi / pm.math.sum(phi, axis=1).reshape((178, 1))

    pm.Multinomial("likelihood", n=1, p=P, observed=Y_data)

    trace = pm.sample(2000, init="jitter+adapt_diag_grad", target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [_b]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 101 seconds.


In [5]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"_b[0, 0]",4.030,4.388,-4.006,12.549,0.044,0.040,10017.0,6287.0,1.0
"_b[0, 1]",-0.553,4.460,-8.916,7.751,0.041,0.057,11840.0,5617.0,1.0
"_b[1, 0]",3.036,1.363,0.418,5.477,0.030,0.022,2022.0,3399.0,1.0
"_b[1, 1]",0.225,1.902,-3.330,3.753,0.037,0.026,2681.0,3522.0,1.0
"_b[2, 0]",-3.619,1.614,-6.691,-0.669,0.028,0.020,3312.0,4885.0,1.0
"_b[2, 1]",1.005,2.208,-2.949,5.246,0.032,0.024,4793.0,4697.0,1.0
"_b[3, 0]",-7.819,3.054,-13.462,-2.050,0.046,0.033,4472.0,4971.0,1.0
"_b[3, 1]",0.686,4.031,-7.107,8.127,0.049,0.044,6833.0,5737.0,1.0
"_b[4, 0]",1.514,0.495,0.623,2.470,0.009,0.006,3231.0,4523.0,1.0
"_b[4, 1]",0.899,0.890,-0.847,2.476,0.016,0.011,3125.0,4210.0,1.0


Coefficients are not the same as BUGS results.

In [6]:
new_data = np.array(
    [1, 12.9, 2, 2.4, 17, 100, 2.8, 2.1, 0.35, 1.6, 5, 1.05, 3, 750]
).reshape((1, 14))
pm.set_data({"X": new_data}, model=m)
ppc = pm.sample_posterior_predictive(trace, model=m)

In [7]:
ppc.posterior_predictive.mean(
    dim=["chain", "draw", "likelihood_dim_0"]
).likelihood.values

array([0.916875, 0.079125, 0.004   ])

Getting the same predicted category, but with slightly different probabilities.

In [8]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

pymc  : 4.0.1
aesara: 2.7.3
numpy : 1.22.4
arviz : 0.12.1
pandas: 1.4.2

